In [1]:
import polars as pl
import numpy as np
from sklearn.neighbors import KernelDensity
from scipy.optimize import minimize
from scipy.special import gammaln
from utils import CDNOW, modified_silverman

import altair as alt
from IPython.display import display_markdown

In [2]:
cdnow = CDNOW(master=False, calwk=273)

# For the Gamma-Gamma model, we need to filter out customers who have made only one purchase.
rfm_data = cdnow.rfm_summary().filter(pl.col('P1X') > 0)

rfm_data_array = rfm_data.select('P1X', 't_x', 'T', 'zbar').collect().to_numpy()
x = rfm_data_array[:,0] # frequency
zbar = rfm_data_array[:,3] / 100 # monetary value
t_x = rfm_data_array[:,1]
T = rfm_data_array[:,2]

The Gamma-Gamma model assumes that there is no relationship between the monetary value and the purchase frequency. We can check this assumption by calculating the correlation between the average spend and the frequency of purchases.

In [3]:
corr_data = rfm_data.select('P1X', 'zbar').collect()
(
    corr_data.corr()
    .with_columns(pl.Series(corr_data.columns).alias("index"))
    .style.tab_header(title="Correlations Between Frequency & Monetary Value")
    .tab_stub(rowname_col="index")
    .fmt_number(decimals=3)
)

# The value of this correlation is close to 0.11, which in practice is considered low enough to proceed with the model.

GT(_tbl_data=shape: (2, 3)
┌──────────┬──────────┬───────┐
│ P1X      ┆ zbar     ┆ index │
│ ---      ┆ ---      ┆ ---   │
│ f64      ┆ f64      ┆ str   │
╞══════════╪══════════╪═══════╡
│ 1.0      ┆ 0.113888 ┆ P1X   │
│ 0.113888 ┆ 1.0      ┆ zbar  │
└──────────┴──────────┴───────┘, _body=<great_tables._gt_data.Body object at 0x13ecfb380>, _boxhead=Boxhead([ColInfo(var='P1X', type=<ColInfoTypeEnum.default: 1>, column_label='P1X', column_align='right', column_width=None), ColInfo(var='zbar', type=<ColInfoTypeEnum.default: 1>, column_label='zbar', column_align='right', column_width=None), ColInfo(var='index', type=<ColInfoTypeEnum.stub: 2>, column_label='index', column_align='left', column_width=None)]), _stub=<great_tables._gt_data.Stub object at 0x13f415a90>, _spanners=Spanners([]), _heading=Heading(title='Correlations Between Frequency & Monetary Value', subtitle=None, preheader=None), _stubhead=None, _source_notes=[], _footnotes=[], _styles=[], _locale=<great_tables._gt_data.Locale object at 0x13ecfb770>, _formats=[<great_tables._gt_data.FormatInfo object at 0x13ecfaf90>], _substitutions=[], _options=Options(table_id=OptionsInfo(scss=False, category='table', type='value', value=None), table_caption=OptionsInfo(scss=False, category='table', type='value', value=None), table_width=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_layout=OptionsInfo(scss=True, category='table', type='value', value='fixed'), table_margin_left=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_margin_right=OptionsInfo(scss=True, category='table', type='px', value='auto'), table_background_color=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_additional_css=OptionsInfo(scss=False, category='table', type='values', value=[]), table_font_names=OptionsInfo(scss=False, category='table', type='values', value=['-apple-system', 'BlinkMacSystemFont', 'Segoe UI', 'Roboto', 'Oxygen', 'Ubuntu', 'Cantarell', 'Helvetica Neue', 'Fira Sans', 'Droid Sans', 'Arial', 'sans-serif']), table_font_size=OptionsInfo(scss=True, category='table', type='px', value='16px'), table_font_weight=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_style=OptionsInfo(scss=True, category='table', type='value', value='normal'), table_font_color=OptionsInfo(scss=True, category='table', type='value', value='#333333'), table_font_color_light=OptionsInfo(scss=True, category='table', type='value', value='#FFFFFF'), table_border_top_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_top_style=OptionsInfo(scss=True, category='table', type='value', value='solid'), table_border_top_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_top_color=OptionsInfo(scss=True, category='table', type='value', value='#A8A8A8'), table_border_right_style=OptionsInfo(scss=True, category='table', type='value', value='none'), table_border_right_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_right_color=OptionsInfo(scss=True, category='table', type='value', value='#D3D3D3'), table_border_bottom_include=OptionsInfo(scss=False, category='table', type='boolean', value=True), table_border_bottom_style=OptionsInfo(scss=True, category='table', type='value', value='solid'), table_border_bottom_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_bottom_color=OptionsInfo(scss=True, category='table', type='value', value='#A8A8A8'), table_border_left_style=OptionsInfo(scss=True, category='table', type='value', value='none'), table_border_left_width=OptionsInfo(scss=True, category='table', type='px', value='2px'), table_border_left_color=OptionsInfo(scss=True, category='table', type='value', value='#D3D3D3'), heading_background_color=OptionsInfo(scss=True, category='heading', type='value', value=None), heading_align=OptionsInfo(scss=True, category='heading', type='value', value=

In [4]:
# Descriptive statistics of the average spend per repeat transaction
summary = rfm_data.select('zbar').with_columns(pl.col('zbar') / 100).describe()
summary

# We note that the distribution of observed individual means is highly skewed to the right.

statistic,zbar
str,f64
"""count""",946.0
"""null_count""",0.0
"""mean""",35.077848
"""std""",30.283506
"""min""",2.99
"""25%""",15.76
"""50%""",27.54
"""75%""",41.79
"""max""",299.63381


Probability density estimate of the sample

In [5]:
m = np.arange(2.5, 301, 2.5) # Average transaction value range

# Apply log transformation for boundary correction
m_log = np.log(m)
zbar_log = np.log(zbar)

bw = modified_silverman(zbar_log)
print('Kernel Smoothing Bandwidth:', bw) 

Kernel Smoothing Bandwidth: 0.18800626075684287


In [6]:
# Estimate the probability density function
# Method 1 - Using sklearn
kde = KernelDensity(kernel='gaussian', bandwidth=bw).fit(zbar_log.reshape(-1,1))
log_density = kde.score_samples(m_log.reshape(-1,1))
f = np.exp(log_density) / m # Transform the density back to the original scale

# Method 2 - Using statsmodels
# import statsmodels.api as sm
# kde = sm.nonparametric.KDEUnivariate(zbar_log)
# kde.fit(kernel='gau', bw=bw)
# f_log = kde.evaluate(m_log)
# f = f_log / m 

The distribution of average spend per (repeat) transaction across the 946 individuals who made a repeat transaction in the calibration period. Each customer’s average
is computed across a (typically very) small number of transactions.

In [7]:
act_dist_plot = (
    alt.Chart(pl.DataFrame({'Average Transaction Value (z)': m, 'f(z)': f}))
    .mark_line().encode(
        x=alt.X('Average Transaction Value (z)', axis=alt.Axis(values=np.arange(0, 301, 50), labelExpr='"$"+datum.value')),
        y=alt.Y('f(z)', scale=alt.Scale(domain=[0, 0.04]))
    )
)

act_dist_plot.properties(
            width=500,
            height=400,
            title='Observed distribution of average transaction values across customers'
).configure_view(stroke=None).configure_axisY(grid=False).configure_axisX(grid=False) 

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


### Parameter Estimation

In [8]:
def gammagamma(x, zbar, guess={'p': 0.01, 'q': 0.01, 'gamma': 0.01}):
        
    def log_likelihood(param):
        p, q, gamma = param[0], param[1], param[2]
        
        ll = gammaln(p*x+q)-gammaln(p*x)-gammaln(q)+q*np.log(gamma)+(p*x-1)*np.log(zbar)+(p*x)*np.log(x)-(p*x+q)*np.log(gamma+x*zbar)
        
        return -np.sum(ll)
    
    bnds = [(1e-6, np.inf) for _ in range(3)]
    
    return minimize(log_likelihood, x0=list(guess.values()), bounds=bnds, method='L-BFGS-B')
        
res = gammagamma(x=x, zbar=zbar)
p, q, gamma = res.x
ll = res.fun

# Sample Parameters
# p = 6.24983547654959
# q = 3.7441106896737
# gamma = 15.4423198312514

display_markdown(f'''$p$ = {p:0.4f}

$q$ = {q:0.4f}

$\\gamma$ = {gamma:0.4f}

Log-Likelihood = {-ll:0.4f}''', raw=True)


$p$ = 6.2506

$q$ = 3.7440

$\gamma$ = 15.4397

Log-Likelihood = -4055.9177

The distribution where the means have been computed across x → ∞ transactions

In [9]:
zeta = np.arange(300) + 1
f_zeta = (p * gamma)**q * zeta**(-q-1) * np.exp(-p*gamma/zeta) / np.exp(gammaln(q))

(
    alt.Chart(pl.DataFrame({'Unobserved mean transaction value (ζ)': zeta, 'f(ζ)': f_zeta}))
    .mark_line().encode(
        x=alt.X('Unobserved mean transaction value (ζ)', axis=alt.Axis(values=np.arange(0, 301, 50), labelExpr='"$"+datum.value')),
        y=alt.Y('f(ζ)', scale=alt.Scale(domain=[0, 0.04]))
    ).properties(
            width=500,
            height=400,
            title='Distribution of the (unobserved) mean transaction value (ζ)'
        ).configure_view(stroke=None).configure_axisY(grid=False).configure_axisX(grid=False) 
)

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


In [10]:
# compute the density of average transaction value

# how many people with each level of x?
repeat_trans_dist = cdnow.rfm_summary().group_by('P1X').agg(pl.len().alias('Count')).sort('P1X').collect().to_numpy()
nx = repeat_trans_dist[1:, 1]
x_trans = repeat_trans_dist[1:, 0]

# compute the density of zbar for each x
y = np.arange(300) + 1
x_trans, y = np.meshgrid(x_trans,y)
a1 = gammaln(p*x_trans+q)-gammaln(p*x_trans)-gammaln(q)
a2 = q*np.log(gamma)
a3 = (p*x_trans-1)*np.log(y)
a4 = (p*x_trans)*np.log(x_trans)
a5 = (p*x_trans+q)*np.log(gamma+y*x_trans)
g1 = np.exp(a1+a2+a3+a4-a5)

# compute the weighted average
g = np.dot(nx, g1.T) / np.sum(nx)

In [11]:
est_dist_plot = (
    alt.Chart(pl.DataFrame({'Average Transaction Value (z)': np.arange(300) + 1, 'f(z)': g}))
    .mark_line(strokeDash=[4,4]).encode(
        x=alt.X('Average Transaction Value (z)', axis=alt.Axis(values=np.arange(0, 301, 50), labelExpr='"$"+datum.value')),
        y=alt.Y('f(z)', scale=alt.Scale(domain=[0, 0.04]))
    )
)

chart = act_dist_plot + est_dist_plot

chart.properties(
            width=500,
            height=400,
            title='Observed versus theoretical distribution of average transaction value across customers'
).configure_view(stroke=None).configure_axisY(grid=False).configure_axisX(grid=False) 

<VegaLite 5 object>

If you see this message, it means the renderer has not been properly enabled
for the frontend that you are using. For more information, see
https://altair-viz.github.io/user_guide/display_frontends.html#troubleshooting


### Computing Conditional Expectations

In [12]:
E_Z = p*gamma/(q-1)

ce = (
    cdnow.rfm_summary()
    .select('ID', 'P1X', 'zbar')   
    .with_columns(((q - 1)/(p * pl.col('P1X') + q - 1)).alias('Weight'))
    .with_columns((pl.col('Weight')*E_Z+(1-pl.col('Weight'))*pl.col('zbar')/100).alias('E(Z|x,zbar)'))
)

ce.collect()

ID,P1X,zbar,Weight,"E(Z|x,zbar)"
i32,u32,f64,f64,f64
1,2,2234.5,0.179989,24.65348
2,1,1177.0,0.30507,18.908823
3,0,0.0,1.0,35.170642
4,0,0.0,1.0,35.170642
5,0,0.0,1.0,35.170642
…,…,…,…,…
2353,0,0.0,1.0,35.170642
2354,5,4492.8,0.080712,44.140463
2355,0,0.0,1.0,35.170642
